In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd
import scanpy as sp

#distribution of distances between 2 types instead of just observation frequency

filepaths = ['/home/imaging_mfish/MERSCOPENAS04_data/human/atlas/merfish_output/202204251522_H1930002Cx58V10200710104_VMSC02501/processed/202204251522_H1930002Cx58V10200710104_VMSC02501.h5ad']
cell_h5ad = sp.read_h5ad(filepaths[0]) 
#obs has class, obsm spatial has location. IDs are matched (so spatial 0 = obs 0)
cell_data = cell_h5ad.obs
cell_data[['x', 'y']] = cell_h5ad.obsm['spatial']

In [ ]:
import timeit

#uniformly random values within 50um circle
radius = np.random.uniform(low=0, high=50, size=1000)  # varies within 50
theta = np.random.uniform(low=0, high=2*np.pi, size=1000)  # angle
rand_circle_points = pd.DataFrame({'x': np.sqrt(radius) * np.cos(theta), 'y':np.sqrt(radius) * np.sin(theta)})
rand_circle_points.loc[len(rand_circle_points)] = [1, 1] #add base case

#create all combinations of subclasses
subclasses = list(test_data.groupby('subclass').groups.keys())[0:2]
unique_subset_pairs = list(itertools.combinations(subclasses, 2))
print(unique_subset_pairs)
#iterate over pairs, then over circle point multipliers to get all possible pairwise distance groupings
def for_timeit(rand_circle_points, cell_data):
    obs_freqs = {}
    for pair in unique_subset_pairs:
        container_df = pd.DataFrame()
        for point in rand_circle_points.itertuples(index=False):
            clust_dists = np.tril(pairwise_distances(cell_data[cell_data['subclass']==pair[0]][['x', 'y']].to_numpy()*point[0], 
                                                     cell_data[cell_data['subclass']==pair[1]][['x', 'y']].to_numpy()*point[1], 
                                                     n_jobs=2)) 
            clust_dists[clust_dists==0] = np.nan
            col_name = str(point[0])+','+str(point[1])
            (foo, bins2, bar) = plt.hist(clust_dists[:].flatten(), bins = 24) #bins = 11 naturally, is that fine? 
            mini_df = pd.DataFrame(pd.Series(bins2), columns=[col_name])
            container_df = pd.concat([container_df, mini_df], axis=1) 
    return container_df
    # calculate one-tailed z-test of observed vs randomized while you're here
    # first filter all values <= 15 (by bin?), then count(*) equivalent to get single number for each iteration
    # z = (sample mean – population mean) / [population standard deviation/sqrt(n)]
    # z = (col=(1,1) - np.mean(col!=(1,1)) / np.std(col!=(1,1)/sqrt(1000))) #check parens here
#     savename = pair[0]+'_'+pair[1]
    # obs_freqs[savename] = [z, savename+'.csv']
#     container_df.to_csv(savename+'.csv') #save pairwise distance grouping
#     return obs_freqs

t = timeit.Timer(lambda: for_timeit(rand_circle_points, cell_data))
print(t.timeit(1))
display(container_df)

[('Astro', 'Chandelier')]


### test/scratch below

cell-cell interactions arising from somatic contact or paracrine signaling, which can be inferred based on contact or proximity between cells that occurred with higher frequency than by random chance (29, 30). 

We examined whether these potential cell-cell interactions were cell-type specific. To this end, we only considered cell types at the subclass level and determined the frequency with which cell-cell contacts (or proximity) were observed between two subclasses of cells. 

Two neighboring cells were considered contacting (or in proximity) if their centroid distance was <15 μm, which is approximately the size of the soma of a single cell in both human and mouse cortex (31). 

We determined how much this frequency was above random chance and how significant this difference was by comparing the observed contact frequency with the expected contact frequencies from spatial permutations. 

To avoid artifacts arising from the laminar organization of cells and spatial variation of cells density (namely, cell types in the regions of higher cell density or with similar laminar distributions can result in a higher contact frequency by random chance), we designed our spatial permutations to only disrupt the spatial relationship between neighboring cells while still preserving the laminar distribution and local density of each cell type


![image.png](attachment:image.png)
Fig S10: 
(A) Schematic of spatial permutation test that determines the significance of interactions between cell types. Two cells were considered contacting if their nucleus centroids were within 15 μm in the imaging plane, which is approximately the size of the cell body of a single neuron. Contact frequency between any two cell types was determined as the observed frequency. Then, spatial localization of each cell was randomized within a radius of 50 μm, unless otherwise mentioned. Expected contact frequency between any two cell types was determined in each permutation and such permutation was performed 1,000 times to obtain the distribution of expected contact frequencies. The significance of observed contact frequency was calculated using one-tailed z-test and P-values were corrected to FDR (false discovery rate) using Benjamini-Hochberg Procedure. 
(B) Spatial map of L2/3 IT cells in a human MTG slice. (Left) Measured spatial map. (Middle and right) Two example spatial maps after spatial permutations described in (A). 
(C) Cortical depth distributions of L2/3 IT cells (light blue) and other cells (light red) in the human MTG slice shown in (B). (Left) Measured cortical depth distributions. (Middle and right) Cortical depth distributions after two example spatial permutations described in (A).

Summary: 
- For Each Z plane, for each specimen
    - Grab all cells within said plane
    - Find all sets of individual cells that are within 15um of one another (maybe go cell by cell? Is there a faster way?)
    - group by cell type and calculate observed frequency of cell type - cell type interaction
    - randomize spatial location of all cells by 50um and recalculate observed frequency (save two of these)
    - do so 1000 times to get a distribution
    - calculate one-tailed z-test of observed vs randomized. Benjamini-Hochberg goes here?
    - plot spatial map of specific cell types of interest pre randomization and post 2 randomizations (this will probably involve saving two randomizations and creating new h5ad files for them, possibly concatting them together so they can all be cirro doritod)
    - plot histogram of cortical depth distro of celltypes of interest vs all other cells pre and post 2 permutations.